<h3> Importation des librairies et sélection des options d'affichage </h3>

In [2]:
import networkx as nx
import gurobipy
from gurobipy import Model, GRB
import matplotlib.pyplot as plt
import sys
import random
import csv

options = {
    "font_size": 20,
    "node_size": 1000,
    "node_color": "white",
    "edgecolors": "black",
    "linewidths": 5,
    "width": 5,
    "with_labels": True,
}

Fonction de géneration d'instance de graphique avec sélection de type de graphe

In [3]:
# Generer des instances de graphes

def generate_graph(graph_type, n_nodes, p=None):
    """
    Génère un graphe selon le type spécifié.
    :param graph_type: Type de graphe ('complete', 'cycle', 'random').
    :param n_nodes: Nombre de nœuds.
    :param p: Probabilité pour les graphes aléatoires (nécessaire pour 'random').
    :return: Graphe généré.
    """
    if graph_type == 'complete':
        G = nx.complete_graph(n_nodes, create_using=nx.DiGraph)
    elif graph_type == 'cycle':
        G = nx.cycle_graph(n_nodes, create_using=nx.DiGraph)
    elif graph_type == 'random':
        G = nx.erdos_renyi_graph(n_nodes, p, directed=True)
    else:
        raise ValueError("Type de graphe non supporté")
    return G


Création des valeurs de capacité et de coût 

In [4]:
def assign_capacities_and_costs(G, capacity_range, cost_range):
    """
    Assigne des capacités et des coûts de routage aux arcs.
    :param G: Graphe.
    :param capacity_range: Intervalle pour les capacités (e.g., (1, 10)).
    :param cost_range: Intervalle pour les coûts (e.g., (1, 5)).
    :return: Graphe avec attributs assignés.
    """
    for u, v in G.edges:
        G[u][v]['capacity'] = random.randint(*capacity_range)
        G[u][v]['cost'] = random.uniform(*cost_range)
    return G

Generation de la demande du traffic 

In [5]:
def generate_traffic_demand(G, S_size, demand_range):
    """
    Génère des sous-ensembles S de nœuds et des demandes de trafic.
    :param G: Graphe.
    :param S_size: Taille de l'ensemble S.
    :param demand_range: Intervalle pour les demandes de trafic (e.g., (1, 20)).
    :return: Ensemble S et vecteur des demandes de trafic.
    """
    nodes = list(G.nodes)
    S = random.sample(nodes, S_size)
    traffic_demand = {}
    for i in S:
        for j in S:
            if i != j:
                traffic_demand[(i, j)] = random.randint(*demand_range)
    return S, traffic_demand

Sauvegarde de l'instance dans un csv

In [6]:
def save_to_csv(G, S, traffic_demand, filename):
    """
    Sauvegarde le graphe, les capacités, les coûts et les demandes de trafic dans un fichier CSV.
    :param G: Graphe.
    :param S: Sous-ensemble des nœuds.
    :param traffic_demand: Dictionnaire des demandes de trafic.
    :param filename: Nom du fichier CSV.
    """
    with open("graph/"+filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Écriture des arcs avec capacités et coûts
        writer.writerow(['Source', 'Target', 'Capacity', 'Cost'])
        for u, v in G.edges:
            writer.writerow([u, v, G[u][v]['capacity'], G[u][v]['cost']])
        
        # Écriture des demandes de trafic
        writer.writerow([])
        writer.writerow(['entry', 'Target', 'Traffic Demand'])
        for (i, j), demand in traffic_demand.items():
            writer.writerow([i, j, demand])
        
        # Écriture de l'ensemble S
        writer.writerow([])
        writer.writerow(['Subset S'])
        writer.writerow(S)

In [7]:
def create_instance(graph_type, n_nodes, capacity_range, cost_range, S_size, demand_range, filename):
    G = generate_graph(graph_type, n_nodes, p=0.3)
    G = assign_capacities_and_costs(G, capacity_range, cost_range)
    S, traffic_demand = generate_traffic_demand(G, S_size, demand_range)
    save_to_csv(G, S, traffic_demand, filename)

    print(f"Instance générée et sauvegardée dans {filename}")
    
    

Exemple utilisation de géneration, attribution des capacités et coût et de la sauvegarde du graphe 

In [8]:
# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres
    graph_type = 'random'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 15  # Nombre de nœuds
    capacity_range = (1, 10)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 5  # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_instance2.csv'  # Nom du fichier CSV

    create_instance(graph_type, n_nodes, capacity_range, cost_range, S_size, demand_range, filename)
    
    

Instance générée et sauvegardée dans graph_instance2.csv


Fonction d'une lecture d'une instance 

In [9]:
def read_instance(filename):
    """
    Lit une instance à partir d'un fichier CSV.
    Args:
        filename (string): nom du fichier csv
    Return:
        Graphe, sous-ensemble S, et demande de trafic  
    """
    G = nx.DiGraph()
    traffic_demand = {}
    S = []
    with open("graph/"+filename, mode='r') as file:
        reader = csv.reader(file)
        section = "arcs"
        for row in reader:
            if not row:
                continue
            if row[0] == "Source":
                continue
            if row[0] == "entry":
                section = "traffic"
            if row[0] == "Subset S":
                section = "subset"
                continue
            if section == "arcs":
                G.add_edge(int(row[0]), int(row[1]), capacity=float(row[2]), cost=float(row[3]))
            elif section =="traffic":
                traffic_demand[(int(row[0]), int(row[1]))] = int(row[2])
            elif section == "subset":
                S = list(map(int, row))
                break
    return G, S, traffic_demand 
    

Fonction pour créer et resoudre un PL pour une fonction objectif donnée 

In [10]:
def solve_linear_program(G, traffic_demand, objective):
    """
    Résout un programme linéaire associé à une fonction objective spécifiée.

    Args:
        G (networkx graph): graphe de l'instance 
        traffic_demand (_type_): valeur de la demande pour le sous ensemble S
        objective (_type_): fonction objective choisie
        
    return:
        solutions de routage et valeur optimale 
    """
    
    model = Model("Routing")
    
    #Variables de décision
    x = {}
    for (i, j), demand in traffic_demand.items():
        for u, v in G.edges:
            x[i, j, u, v] = model.addVar(lb=0, ub=1, name=f"x_{i}_{j}_{u}_{v}")
    
    #Contraintes de conservation de flux
    for (i, j), demand in traffic_demand.items():
        for node in G.nodes:
            inflow = sum(x[(i, j, u, v)] for u, v in G.in_edges(node))
            outflow = sum(x[(i, j, u, v)] for u, v, in G.out_edges(node))
            if node == i:
                model.addConstr(outflow - inflow == 1, name=f"flow_src_{i}_{j}_{node}")
            elif node == j:
                model.addConstr(inflow - outflow == 1, name=f"flow_dst_{i}_{j}_{node}")
            else:
                model.addConstr(inflow - outflow == 0, name=f"flow_mid_{i}_{j}_{node}")

    # Constrainte de capacité 
    for u, v in G.edges:    
        model.addConstr(
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) <= G[u][v]['capacity'],
            name=f"capacity_{u}_{v}"
        )
    
    # Objectifs
    if objective == "cost":
        model.setObjective(
            sum(x[(i, j, u, v)] * demand * G[u][v]['cost'] for (i, j), demand in traffic_demand.items() for u, v in G.edges),
            GRB.MINIMIZE
        )
    elif objective == "max_utlilization":
        utilisation = model.addVar(lb=0, ub=GRB.INFINITY, name="max_utilisation")
        for u, v in G.edges:
            model.addConstr(
                utilisation >=
                sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity'],
                name=f"utilisation_{u}_{v}"
            )
        model.setObjective(utilisation, GRB.MINIMIZE)
    elif objective == "avg_utilisation":
        avg_utilisation = sum(
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity']
            for u, v in G.edges
        ) / G.number_of_edges()
        model.setObjective(avg_utilisation, GRB.MINIMIZE)
    else:
        raise ValueError("Fonction objectif non reconnue")

    #Résolution
    model.optimize()
    
    # Débogage en cas d'infaisabilité
    if model.status == GRB.INFEASIBLE:
        print("Modèle infaisable. Analyse des conflits...")
        model.computeIIS()
        model.write("infeasible.ilp")
        print("Conflits enregistrés dans infeasible.ilp")
        return None
    
    #Extraction de la solution
    if model.status == GRB.OPTIMAL:
        solution = {key: var.x for key, var in x.items() if var.x > 0}
        return solution, model.ObjVal
    else:
        raise Exception("Pas de solution optimale trouvée")
    

Fonction de test de la résolution d'une instance de graphe

In [14]:
def test_program(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, objective, filename):
    """
    Teste le programme de résolution de PL sur un type de graphe

    Args:
        graph_type (string): type de graphe 
        n_nodes (int): nombre de node dans V
        S_size (int): nombre de node dans S
        demand_range (tuple): tuple de 2 int pour le range des valeurs de demande 
        capacity_range (tuple): tuple de 2 int pour le range des valeurs de capacité
        cost_range (tuple): tuple de 2 int pour le range des valeurs de capacité
        objective (string): nom de la fonction objective choisi (cost, max_utlilization, avg_utlilization)
    """
    G = generate_graph(graph_type, n_nodes, 0.3)
    G = assign_capacities_and_costs(G, capacity_range, cost_range)
    S, traffic_demand = generate_traffic_demand(G, S_size, demand_range)
    save_to_csv(G, S, traffic_demand, filename)
    solution, optimal_value = solve_linear_program(G, traffic_demand, objective)
    print(f"Solution optimale pour {objective}:", solution)
    print(f"Valeur optimale pour {objective}: {optimal_value}")

Graphe de type Random

In [17]:
# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres
    graph_type = 'random'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 15  # Nombre de nœuds
    capacity_range = (10, 20)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 5  # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_random.csv'  # Nom du fichier CSV
    objective = "cost" # Choix de la fonction objective 
    test_program(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, objective, filename)
    
    

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 365 rows, 1300 columns and 3900 nonzeros
Model fingerprint: 0x09e75efc
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 20 rows and 0 columns
Presolve time: 0.03s
Presolved: 345 rows, 1300 columns, 3738 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.800000e+01   0.000000e+00      0s
     151    8.8330533e+02   0.000000e+00   0.000000e+00      0s

Solved in 151 iterations and 0.05 seconds (0.00 work units)
Optimal objective  8.833053342e+02
Solution optimale pour cost: {(2, 4, 2, 6): 1.0, (2, 4, 5, 4): 1.0, (2, 4, 6, 5): 1.0, (2, 0, 2, 0)

Graphe de type complet

In [20]:
# Exemple d'utilisation pour un graphe complet
if __name__ == "__main__":
    # Paramètres pour le graphe complet
    graph_type = 'complete'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 8  # Nombre de nœuds
    capacity_range = (10, 20)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 3 # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_complete.csv'  # Nom du fichier CSV
    objective = "cost" # Choix de la fonction objective 
    test_program(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, objective, filename)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 104 rows, 336 columns and 1008 nonzeros
Model fingerprint: 0xe1a99751
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 6 rows and 0 columns
Presolve time: 0.02s
Presolved: 98 rows, 336 columns, 924 nonzeros



Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.200000e+01   0.000000e+00      0s
      32    1.8620093e+02   0.000000e+00   0.000000e+00      0s

Solved in 32 iterations and 0.05 seconds (0.00 work units)
Optimal objective  1.862009310e+02
Solution optimale pour cost: {(2, 3, 2, 3): 0.8235294117647058, (2, 3, 2, 7): 0.17647058823529416, (2, 3, 7, 3): 0.17647058823529416, (2, 4, 2, 7): 1.0, (2, 4, 7, 4): 1.0, (3, 2, 3, 2): 1.0, (3, 4, 0, 4): 0.2666666666666667, (3, 4, 3, 0): 0.2666666666666667, (3, 4, 3, 4): 0.7333333333333333, (4, 2, 4, 2): 0.8888888888888888, (4, 2, 4, 5): 0.11111111111111116, (4, 2, 5, 2): 0.11111111111111116, (4, 3, 4, 3): 0.9444444444444444, (4, 3, 4, 6): 0.05555555555555558, (4, 3, 6, 3): 0.05555555555555558}
Valeur optimale pour cost: 186.20093097726397


Graphe de type cycle

In [44]:
# Exemple d'utilisation pour un graphe cycle
if __name__ == "__main__":
    # Paramètres pour le graphe cycle
    graph_type = 'cycle'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 6  # Nombre de nœuds
    capacity_range = (40, 60)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 3 # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_cycle.csv'  # Nom du fichier CSV
    objective = "cost" # Choix de la fonction objective 
    test_program(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, objective, filename)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 42 rows, 36 columns and 108 nonzeros
Model fingerprint: 0x31c8a02e
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [3e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 42 rows and 36 columns
Presolve time: 0.08s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3386428e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.10 seconds (0.00 work units)
Optimal objective  4.338642797e+02
Solution optimale pour cost: {(5, 2, 0, 1): 1.0, (5, 2, 1, 2): 1.0, (5, 2, 5, 0): 1.0, (5, 3, 0, 1): 1.0, (5, 3, 1, 2): 1.0, (5, 3, 2, 3): 1.0, (5, 3, 5, 0): 1.0, (2, 5, 2, 3): 1.

Méthode de la somme pondérée

In [45]:
# Implémentation des méthodes d'obtention de solutions efficaces
def weighted_sum_method(G, traffic_demand, weights):
    """
    Implémente la méthode des sommes pondées pour le routage multiobjectif.
    :param G: Graphe orienté.
    :param traffic_demand: Dictionnaire des demandes de trafic.
    :param weights: Poids pour les objectifs (liste de trois poids).
    :return: Solution optimale et valeur des objectifs.
    """
    model = Model("WeightedSumRouting")

    # Variables de décision
    x = {}
    for (i, j), demand in traffic_demand.items():
        for u, v in G.edges:
            x[(i, j, u, v)] = model.addVar(lb=0, ub=1, name=f"x_{i}_{j}_{u}_{v}")

    # Contraintes de conservation de flux
    for (i, j), demand in traffic_demand.items():
        for node in G.nodes:
            inflow = sum(x[(i, j, u, v)] for u, v in G.in_edges(node))
            outflow = sum(x[(i, j, u, v)] for u, v in G.out_edges(node))
            if node == i:
                model.addConstr(outflow - inflow == 1, name=f"flow_src_{i}_{j}_{node}")
            elif node == j:
                model.addConstr(inflow - outflow == 1, name=f"flow_dst_{i}_{j}_{node}")
            else:
                model.addConstr(inflow - outflow == 0, name=f"flow_mid_{i}_{j}_{node}")

    # Contraintes de capacité
    for u, v in G.edges:
        model.addConstr(
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) <= G[u][v]['capacity'],
            name=f"capacity_{u}_{v}"
        )

    # Objectifs individuels
    cost = sum(
        x[(i, j, u, v)] * demand * G[u][v]['cost'] for (i, j), demand in traffic_demand.items() for u, v in G.edges
    )

    max_utilization = model.addVar(lb=0, ub=GRB.INFINITY, name="max_utilization")
    for u, v in G.edges:
        model.addConstr(
            max_utilization >=
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity'],
            name=f"utilization_{u}_{v}"
        )

    avg_utilization = sum(
        sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity']
        for u, v in G.edges
    ) / G.number_of_edges()

    # Fonction objectif pondérée
    model.setObjective(weights[0] * cost + weights[1] * max_utilization + weights[2] * avg_utilization, GRB.MINIMIZE)

    # Résolution
    model.optimize()

    # Extraction des résultats
    if model.status == GRB.OPTIMAL:
        solution = {key: var.x for key, var in x.items() if var.x > 0}
        return solution, cost.getValue(), max_utilization.x, avg_utilization.getValue()
    else:
        raise Exception("Modèle non résolu (statut: {model.status})")

# Exemple d'utilisation
def test_weighted_sum():
    # Créer un graphe simple
    G = nx.DiGraph()
    G.add_edge(0, 1, capacity=5, cost=2)
    G.add_edge(1, 2, capacity=5, cost=3)
    G.add_edge(0, 2, capacity=1, cost=1)

    traffic_demand = {(0, 2): 3, (0, 1): 2}
    weights = [0.5, 0.3, 0.2]  # Pondérations des objectifs

    solution, cost, max_util, avg_util = weighted_sum_method(G, traffic_demand, weights)
    print("Solution optimale :", solution)
    print("Coût total :", cost)
    print("Utilisation maximale :", max_util)
    print("Utilisation moyenne :", avg_util)

# Lancer le test
test_weighted_sum()


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 7 columns and 27 nonzeros
Model fingerprint: 0x0c7b1151
Coefficient statistics:
  Matrix range     [4e-01, 3e+00]
  Objective range  [3e-01, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 12 rows and 7 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.9466667e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.07 seconds (0.00 work units)
Optimal objective  7.946666667e+00
Solution optimale : {(0, 2, 0, 1): 0.6666666666666667, (0, 2, 0, 2): 0.3333333333333333, (0, 2, 1, 2): 0.6666666666666667, (0, 1, 0, 1): 1.0}
Coût total : 15.0
Utilisation maximale :

Epsilon contrainte 

In [46]:
def epsilon_constraint_method(G, traffic_demand, primary_objective, secondary_objective, epsilon):
    """
    Implémente la méthode des ε-contraintes pour le routage multiobjectif.
    :param G: Graphe orienté.
    :param traffic_demand: Dictionnaire des demandes de trafic.
    :param primary_objective: Objectif principal ('cost', 'max_utilization', 'avg_utilization').
    :param secondary_objective: Objectif secondaire pour les contraintes.
    :param epsilon: Valeur limite pour l'objectif secondaire.
    :return: Solution optimale et valeur des objectifs.
    """
    model = Model("EpsilonConstraintRouting")

    # Variables de décision
    x = {}
    for (i, j), demand in traffic_demand.items():
        for u, v in G.edges:
            x[(i, j, u, v)] = model.addVar(lb=0, ub=1, name=f"x_{i}_{j}_{u}_{v}")

    # Contraintes de conservation de flux
    for (i, j), demand in traffic_demand.items():
        for node in G.nodes:
            inflow = sum(x[(i, j, u, v)] for u, v in G.in_edges(node))
            outflow = sum(x[(i, j, u, v)] for u, v in G.out_edges(node))
            if node == i:
                model.addConstr(outflow - inflow == 1, name=f"flow_src_{i}_{j}_{node}")
            elif node == j:
                model.addConstr(inflow - outflow == 1, name=f"flow_dst_{i}_{j}_{node}")
            else:
                model.addConstr(inflow - outflow == 0, name=f"flow_mid_{i}_{j}_{node}")

    # Contraintes de capacité
    for u, v in G.edges:
        model.addConstr(
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) <= G[u][v]['capacity'],
            name=f"capacity_{u}_{v}"
        )

    # Objectifs individuels
    cost = sum(
        x[(i, j, u, v)] * demand * G[u][v]['cost'] for (i, j), demand in traffic_demand.items() for u, v in G.edges
    )

    max_utilization = model.addVar(lb=0, ub=GRB.INFINITY, name="max_utilization")
    for u, v in G.edges:
        model.addConstr(
            max_utilization >=
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity'],
            name=f"utilization_{u}_{v}"
        )

    avg_utilization = sum(
        sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity']
        for u, v in G.edges
    ) / G.number_of_edges()

    # Contraintes sur l'objectif secondaire
    if secondary_objective == 'cost':
        model.addConstr(cost <= epsilon, name="epsilon_cost")
    elif secondary_objective == 'max_utilization':
        model.addConstr(max_utilization <= epsilon, name="epsilon_max_utilization")
    elif secondary_objective == 'avg_utilization':
        model.addConstr(avg_utilization <= epsilon, name="epsilon_avg_utilization")
    else:
        raise ValueError("Objectif secondaire non reconnu")

    # Objectif principal
    if primary_objective == 'cost':
        model.setObjective(cost, GRB.MINIMIZE)
    elif primary_objective == 'max_utilization':
        model.setObjective(max_utilization, GRB.MINIMIZE)
    elif primary_objective == 'avg_utilization':
        model.setObjective(avg_utilization, GRB.MINIMIZE)
    else:
        raise ValueError("Objectif principal non reconnu")

    # Résolution
    model.optimize()

    # Extraction des résultats
    if model.status == GRB.OPTIMAL:
        solution = {key: var.x for key, var in x.items() if var.x > 0}
        return solution, cost.getValue(), max_utilization.x, avg_utilization.getValue()
    else:
        raise Exception("Modèle non résolu (statut: {model.status})")

# Exemple d'utilisation
def test_epsilon_constraint():
    # Créer un graphe simple
    G = nx.DiGraph()
    G.add_edge(0, 1, capacity=5, cost=2)
    G.add_edge(1, 2, capacity=5, cost=3)
    G.add_edge(0, 2, capacity=1, cost=1)

    traffic_demand = {(0, 2): 3, (0, 1): 2}
    primary_objective = 'cost'
    secondary_objective = 'max_utilization'
    epsilon = 2.5

    solution, cost, max_util, avg_util = epsilon_constraint_method(G, traffic_demand, primary_objective, secondary_objective, epsilon)
    print("Solution optimale :", solution)
    print("Coût total :", cost)
    print("Utilisation maximale :", max_util)
    print("Utilisation moyenne :", avg_util)

# Lancer le test
test_epsilon_constraint()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 7 columns and 28 nonzeros
Model fingerprint: 0x6be883ec
Coefficient statistics:
  Matrix range     [4e-01, 3e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 13 rows and 7 columns
Presolve time: 0.05s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.08 seconds (0.00 work units)
Optimal objective  1.500000000e+01
Solution optimale : {(0, 2, 0, 1): 0.6666666666666667, (0, 2, 0, 2): 0.3333333333333333, (0, 2, 1, 2): 0.6666666666666667, (0, 1, 0, 1): 1.0}
Coût total : 15.0
Utilisation maximale :

Méthode lexicographique

In [47]:
def lexicographic_method(G, traffic_demand, objectives):
    """
    Implémente la méthode d'optimisation lexicographique.
    :param G: Graphe orienté.
    :param traffic_demand: Dictionnaire des demandes de trafic.
    :param objectives: Liste d'objectifs par ordre de priorité ('cost', 'max_utilization', 'avg_utilization').
    :return: Solution optimale et valeurs des objectifs.
    """
    model = Model("LexicographicRouting")

    # Variables de décision
    x = {}
    for (i, j), demand in traffic_demand.items():
        for u, v in G.edges:
            x[(i, j, u, v)] = model.addVar(lb=0, ub=1, name=f"x_{i}_{j}_{u}_{v}")

    # Contraintes de conservation de flux
    for (i, j), demand in traffic_demand.items():
        for node in G.nodes:
            inflow = sum(x[(i, j, u, v)] for u, v in G.in_edges(node))
            outflow = sum(x[(i, j, u, v)] for u, v in G.out_edges(node))
            if node == i:
                model.addConstr(outflow - inflow == 1, name=f"flow_src_{i}_{j}_{node}")
            elif node == j:
                model.addConstr(inflow - outflow == 1, name=f"flow_dst_{i}_{j}_{node}")
            else:
                model.addConstr(inflow - outflow == 0, name=f"flow_mid_{i}_{j}_{node}")

    # Contraintes de capacité
    for u, v in G.edges:
        model.addConstr(
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) <= G[u][v]['capacity'],
            name=f"capacity_{u}_{v}"
        )

    # Objectifs individuels
    cost = sum(
        x[(i, j, u, v)] * demand * G[u][v]['cost'] for (i, j), demand in traffic_demand.items() for u, v in G.edges
    )

    max_utilization = model.addVar(lb=0, ub=GRB.INFINITY, name="max_utilization")
    for u, v in G.edges:
        model.addConstr(
            max_utilization >=
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity'],
            name=f"utilization_{u}_{v}"
        )

    avg_utilization = sum(
        sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity']
        for u, v in G.edges
    ) / G.number_of_edges()

    # Résolution lexicographique
    results = {}
    for objective in objectives:
        if objective == 'cost':
            model.setObjective(cost, GRB.MINIMIZE)
        elif objective == 'max_utilization':
            model.setObjective(max_utilization, GRB.MINIMIZE)
        elif objective == 'avg_utilization':
            model.setObjective(avg_utilization, GRB.MINIMIZE)
        else:
            raise ValueError("Objectif non reconnu")

        model.optimize()

        if model.status == GRB.OPTIMAL:
            results[objective] = model.objVal

            # Ajouter les contraintes pour figer l'objectif courant
            if objective == 'cost':
                model.addConstr(cost <= model.objVal, name="fix_cost")
            elif objective == 'max_utilization':
                model.addConstr(max_utilization <= model.objVal, name="fix_max_utilization")
            elif objective == 'avg_utilization':
                model.addConstr(avg_utilization <= model.objVal, name="fix_avg_utilization")
        else:
            raise Exception(f"Modèle non résolu pour l'objectif {objective} (statut: {model.status})")

    # Extraction des résultats finaux
    solution = {key: var.x for key, var in x.items() if var.x > 0}
    return solution, results

# Exemple d'utilisation
def test_lexicographic_method():
    # Créer un graphe simple
    G = nx.DiGraph()
    G.add_edge(0, 1, capacity=5, cost=2)
    G.add_edge(1, 2, capacity=5, cost=3)
    G.add_edge(0, 2, capacity=1, cost=1)

    traffic_demand = {(0, 2): 3, (0, 1): 2}
    objectives = ['cost', 'max_utilization', 'avg_utilization']

    solution, results = lexicographic_method(G, traffic_demand, objectives)
    print("Solution optimale :", solution)
    print("Résultats lexicographiques :", results)

# Lancer le test
test_lexicographic_method()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 7 columns and 27 nonzeros
Model fingerprint: 0x3eebf852
Coefficient statistics:
  Matrix range     [4e-01, 3e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 12 rows and 7 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.05 seconds (0.00 work units)
Optimal objective  1.500000000e+01
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread co